<a href="https://colab.research.google.com/github/Ramyanee/fake_news/blob/master/fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install kaggle==1.5.6 
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/  #make sure file name is same as the one you uploaded
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

Saving kaggle.json to kaggle.json
ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
allen-institute-for-ai/CORD-19-research-challenge           COVID-19 Open Research Dataset Challenge (CORD-19)    2GB  2020-05-02 20:35:01          65140  
roche-data-science-coalition/uncover                        UNCOVER COVID-19 Challenge                          142MB  2020-04-29 22:47:23           8263  
jessemostipak/animal-crossing                               Animal Crossing                                     754KB  2020-05-04 15:30:32            346  
mrmorj/political-advertisements-from-facebook               Political Advertisements from Facebook              237MB  2020-05-05 18:53:51             40  
mrisdal/married-at-first-sight

In [0]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset


 81% 33.0M/41.0M [00:00<00:00, 50.4MB/s]
100% 41.0M/41.0M [00:00<00:00, 92.3MB/s]


In [0]:
!unzip \*.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [0]:
import pandas as pd

In [0]:
d1 = pd.read_csv('True.csv')

In [0]:
d0 = pd.read_csv('Fake.csv')

In [0]:
d1.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [0]:
d0.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [0]:
d1['y']= 1
d0['y']=0

In [0]:
d0.describe()

,y
count,23481.0
mean,0.0
std,0.0
min,0.0
25%,0.0
50%,0.0
75%,0.0
max,0.0


In [0]:
data = pd.concat([d0,d1]).sort_values(by=['title'])

In [0]:
data.title


7786     \r\r\r\r\r\r\r\nDonald Trump Gets His A** Hand...
1050     \r\r\r\r\r\r\r\nEx-GOP Congressman Shreds Fell...
3930     \r\r\r\r\r\r\r\nTrump Gets STOMPED For Whining...
1594     \r\r\r\r\r\r\r\nWATCH: Democratic Rep Delivers...
5161      #AfterTrumpImplodes Hashtag Hilariously Imagi...
                               ...                        
14235    “YOU’RE HIRED!” Trump Pulls Unemployed Vet Fro...
16696    “YOU’RE HIRED!” Trump Pulls Unemployed Vet Fro...
11729    “Your rights are NOT superior to mine!” JUDGE ...
15113    “You’re Not Welcome!” Obama As Welcome At Rose...
21428    “You’re Not Welcome!” Obama As Welcome At Rose...
Name: title, Length: 44898, dtype: object

In [0]:
from sklearn.model_selection import train_test_split as tts
title_train, title_test,text_train, text_test,subject_train, subject_test,date_train, date_test, ytrain, ytest = tts(data.title,data.text,data.subject, data.date,data.y, test_size = 0.33)

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer as tk
from tensorflow.keras.preprocessing.sequence import pad_sequences as pd_sq

In [0]:

MAX_VOCAB = 20000
def preprocess_sentence(sentence):
  tknzr = tk(num_words = MAX_VOCAB)
  s= tknzr.fit_on_texts(tknzr.sequences_to_texts(sentence))
  sq_sentence = tknzr.texts_to_sequences(tknzr.sequences_to_texts(sentence))
  word2index = tknzr.word_index
  v= len(word2index)
  return sq_sentence,v

def pad(s1,s2):
  s_tr = pd_sq(s1)
  dim = s_tr.shape #N*T
  T = dim[1]
  N = dim[0]
  s_ts = pd_sq(s2, maxlen = T)
  return s_tr,s_ts,dim

In [0]:

title_train, v_title_train = preprocess_sentence(title_train)
title_test, v_title_test = preprocess_sentence(title_test)
text_train, v_text_train = preprocess_sentence(text_train)
text_test, v_text_test = preprocess_sentence(text_test)
subject_train, v_subject_train = preprocess_sentence(subject_train)
subject_test, v_subject_test = preprocess_sentence(subject_test)
date_train, v_date_train = preprocess_sentence(date_train)
date_test, v_date_test = preprocess_sentence(date_test)

In [0]:
title_train, title_test, title_dim = pad(title_train,title_test)
text_train, text_test, text_dim = pad(text_train, text_test)
subject_train, subject_test, subject_dim = pad(subject_train, subject_test)
date_train, date_test, date_dim = pad(date_train, date_test)

In [0]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM, Embedding
from tensorflow.keras.models import Model
#embedding dimensionality
D = 20
# hidden state dimensionality
M =15
def train_test_plot(T, V, data_train, data_test):
  i = Input(shape = (T,) )
  x=  Embedding(V+1,D)(i)
  x=LSTM(M, return_sequences=True)(x)
  x=GlobalMaxPooling1D()(x)
  x= Dense(1, activation='sigmoid')(x)

  model = Model(i,x)
  model.compile(
      loss= 'binary_crossentropy',
      optimizer = 'adam',
      metrics = ['accuracy']
  )
  r  = model.fit(
      data_train,
      ytrain,
      epochs = 10,
      validation_data = (data_test, ytest)
  )

  plt.plot(r.history['loss'], label = 'loss ')
  plt.plot(r.history['val_loss'], label = 'val_loss')
  plt.legend()

In [0]:
train_test_plot(title_dim[1],v_title_train,title_train, title_test)

In [0]:
train_test_plot(text_dim[1],v_text_train,text_train, text_test)

In [0]:
train_test_plot(subject_dim[1],v_subject_train,subject_train, subject_test)

In [0]:
train_test_plot(date_dim[1],v_date_train,date_train, date_test)